In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam

In [3]:
df = pd.read_csv("/Users/work/Desktop/DataScience/Projects/textgen-lstm-gpt2/data/freecodecamp_casual_chatroom.csv")

df.head()

/var/folders/ml/s9qbdr0x6z7588vxmc4pgrfr0000gq/T/ipykernel_91801/173248410.py:1: DtypeWarning: Columns (3,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/work/Desktop/DataScience/Projects/textgen-lstm-gpt2/data/freecodecamp_casual_chatroom.csv")


,Unnamed: 0,editedAt,fromUser,fromUser.avatarUrl,fromUser.avatarUrlMedium,fromUser.avatarUrlSmall,fromUser.displayName,fromUser.gv,fromUser.id,fromUser.removed,...,issues,mentions,meta,readBy,sent,status,text,unread,urls,v
0,0,NaN,NaN,NaN,https://avatars2.githubusercontent.com/u/21996...,https://avatars2.githubusercontent.com/u/21996...,Charles Watson,3.0,546fd99cdb8155e6700d6ec8,NaN,...,[],[],[],22,2014-12-31T23:01:35.647Z,NaN,no legumes either,False,[],1
1,1,NaN,NaN,NaN,https://avatars0.githubusercontent.com/u/10361...,https://avatars0.githubusercontent.com/u/10361...,janetwalters008,NaN,54a47e0cdb8155e6700e486e,NaN,...,[],[],[],21,2014-12-31T23:02:51.600Z,NaN,That bullet proof coffee sounds insane.,False,[],1
2,2,NaN,NaN,NaN,https://avatars0.githubusercontent.com/u/10361...,https://avatars0.githubusercontent.com/u/10361...,janetwalters008,NaN,54a47e0cdb8155e6700e486e,NaN,...,[],[],[],21,2014-12-31T23:03:14.221Z,NaN,That guy has huge eyes.,False,[],1
3,3,NaN,NaN,NaN,https://avatars2.githubusercontent.com/u/21996...,https://avatars2.githubusercontent.com/u/21996...,Charles Watson,3.0,546fd99cdb8155e6700d6ec8,NaN,...,[],"[{'userIds': [], 'userId': '54a47e0cdb8155e670...",[],22,2014-12-31T23:03:20.182Z,NaN,@janetwalters008 It is. but it works. some peo...,False,[],1
4,4,NaN,NaN,NaN,https://avatars0.githubusercontent.com/u/42194...,https://avatars0.githubusercontent.com/u/42194...,Paul Gilliam,3.0,54a44bbbdb8155e6700e47de,NaN,...,[],[],[],22,2014-12-31T23:03:38.388Z,NaN,They guy that came up with the idea is kind of...,False,[],1


In [4]:
df = df[df["text"].notna()]
texts = df["text"].astype(str).tolist()
texts = texts[:int(len(texts)*0.001)]

In [5]:
corpus = " ".join(texts).lower()

In [6]:
chars = sorted(list(set(corpus)))
print("Unique characters:", len(chars))
print(chars[:100])

Unique characters: 79
['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', 'å', '–', '—', '‘', '’', '“', '”', '…']


In [7]:
char_freq = Counter(corpus)

min_freq = 1000
valid_chars = {char for char, freq in char_freq.items() if freq >= min_freq}

clean_corpus = ''.join(c for c in corpus if c in valid_chars)



In [8]:
valid_chars = sorted(list(set(clean_corpus)))

char_to_idx = {c: i for i, c in enumerate(valid_chars)}
idx_to_char = {i: c for i, c in enumerate(valid_chars)}

print(char_to_idx['a'], idx_to_char[char_to_idx['a']])


6 a


In [9]:
print("Unique characters:", len(valid_chars))


Unique characters: 29


In [10]:
maxlen = 40
step = 3

sequences = []
next_chars = []

for i in range(0, len(clean_corpus) - maxlen, step):
    sequences.append(clean_corpus[i: i + maxlen])
    next_chars.append(clean_corpus[i + maxlen])

print("Number of sequences:", len(sequences))
print("Example X:", sequences[0])
print("Example y:", next_chars[0])


Number of sequences: 100741
Example X: no legumes either that bullet proof coff
Example y: e


In [11]:
num_samples = len(sequences)
maxlen = len(sequences[0])

X = np.zeros((num_samples, maxlen), dtype=np.int32)
y = np.zeros(num_samples, dtype=np.int32)

for i, seq in enumerate(sequences):
    X[i] = [char_to_idx[c] for c in seq]
    y[i] = char_to_idx[next_chars[i]]

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Example X[0]:", X[0])
print("Example y[0]:", y[0])

Shape of X: (100741, 40)
Shape of y: (100741,)
Example X[0]: [19 20  0 17 10 12 25 18 10 23  0 10 14 24 13 10 22  0 24 13  6 24  0  7
 25 17 17 10 24  0 21 22 20 20 11  0  8 20 11 11]
Example y[0]: 10


In [12]:
vocab_size = len(char_to_idx) 
embedding_dim = 32                
lstm_units = 128

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),
    LSTM(lstm_units, return_sequences=False),
    Dropout(0.2),
    Dense(vocab_size, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

model.summary()

/Users/work/PycharmProjects/AI_HW4/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = model.fit(X, y, batch_size=64, epochs=20, validation_split=0.1)


Epoch 1/20
1417/1417 ━━━━━━━━━━━━━━━━━━━━ 40s 28ms/step - accuracy: 0.4426 - loss: 1.9081 - val_accuracy: 0.4630 - val_loss: 1.8433
Epoch 2/20
1417/1417 ━━━━━━━━━━━━━━━━━━━━ 38s 27ms/step - accuracy: 0.4634 - loss: 1.8315 - val_accuracy: 0.4788 - val_loss: 1.7960
Epoch 3/20
1417/1417 ━━━━━━━━━━━━━━━━━━━━ 39s 28ms/step - accuracy: 0.4831 - loss: 1.7629 - val_accuracy: 0.4858 - val_loss: 1.7575
Epoch 4/20
1417/1417 ━━━━━━━━━━━━━━━━━━━━ 40s 29ms/step - accuracy: 0.4980 - loss: 1.7140 - val_accuracy: 0.4974 - val_loss: 1.7302
Epoch 5/20
1417/1417 ━━━━━━━━━━━━━━━━━━━━ 40s 28ms/step - accuracy: 0.5089 - loss: 1.6670 - val_accuracy: 0.5056 - val_loss: 1.7144
Epoch 6/20
1417/1417 ━━━━━━━━━━━━━━━━━━━━ 40s 29ms/step - accuracy: 0.5184 - loss: 1.6321 - val_accuracy: 0.5088 - val_loss: 1.6982
Epoch 7/20
1417/1417 ━━━━━━━━━━━━━━━━━━━━ 40s 28ms/step - accuracy: 0.5278 - loss: 1.5966 - val_accuracy: 0.5132 - val_loss: 1.6921
Epoch 8/20
1417/1417 ━━━━━━━━━━━━━━━━━━━━ 43s 30ms/step - accuracy: 0.5359 -

In [15]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_text(model, seed_text, char_to_idx, idx_to_char, maxlen=40, length=300, temperature=1.0):
    generated = seed_text
    input_seq = seed_text[-maxlen:]
    
    for _ in range(length):
        input_indices = [char_to_idx.get(c, 0) for c in input_seq]
        input_padded = np.zeros((1, maxlen), dtype=np.int32)
        input_padded[0, -len(input_indices):] = input_indices

        preds = model.predict(input_padded, verbose=0)[0]
        
        next_index = sample(preds, temperature)
        next_char = idx_to_char[next_index]

        generated += next_char
        input_seq = generated[-maxlen:]

    return generated

In [20]:
seed = "what are you working on"
generated_text = generate_text(model, seed, char_to_idx, idx_to_char, maxlen=maxlen, length=20, temperature=0.8)

print("Seed:", seed)
print("\nGenerated:\n", generated_text)

Seed: what are you working on

Generated:
 what are you working on the dorumant when y
